In [ ]:
import numpy as np
from ctypes import *
import socket
#import errno
import time

In [ ]:
NBUF = 16
BLK_SIZE = 1024*16*1000
DATA_SIZE = BLK_SIZE * NBUF
HOST = '0.0.0.0'
PORT = 5000
FPGA_BLK_SZ = (8192 + 64) * 800000
FPGA_BLK_NB = 30
FPGA_SIZE = 2**30*185
FPGA_DATA_SIZE = FPGA_BLK_SZ * FPGA_BLK_NB
MASK_BLK_SZ = 100000

intensity = np.memmap('/dev/hugepages/fpga0.bin', dtype='float32', shape=(DATA_SIZE))
blk_ptr = cast(c_void_p(intensity.ctypes.data + DATA_SIZE * 4), POINTER(c_long))
clk_counter = np.zeros(8, dtype='u1')

fpga_data = np.memmap('/mnt/fpga0', dtype='u1', shape=(FPGA_SIZE))

def cvt_counter(data):
    clk_counter[:5] = data
    return clk_counter.view(dtype='u8')[0]

In [ ]:
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen(1)

    while True:
        conn, addr = s.accept()
        print('Connected by', addr)

        last_blk = blk_ptr[0]
        while True:
            while last_blk == blk_ptr[0]:
                time.sleep(0.01)

            data = intensity[BLK_SIZE*last_blk:BLK_SIZE*(last_blk+1)]
            hdr_id = blk_ptr[last_blk + 1]
            p = FPGA_BLK_SZ * hdr_id
            try:
                conn.sendall(fpga_data[p:p+64])
                conn.sendall(data)
            except socket.error as e:
#                if e.errno != errno.ECONNRESET:
#                    raise
                print('Connection closed for', addr)
                break

            print(f'Block #{last_blk} sent, header id= {hdr_id}, counter={fpga_data[p:p+5]}:{cvt_counter(fpga_data[p:p+5])}')
            last_blk = (last_blk + 1) % NBUF

In [ ]:
fpga_data[:64]

In [ ]:
for n in range(16):
    p = (8192 + 64) * 0 + FPGA_BLK_SZ * n
    print(n, cvt_counter(fpga_data[p:p+5]), fpga_data[p:p+5], fpga_data[p+8:p+13], fpga_data[p+55:p+64].tobytes())

In [ ]:
vhex = np.vectorize(hex)

In [ ]:
for m in range(10):
#    for n in range(FPGA_BLK_NB):
    for n in range(1,2):
        p = FPGA_DATA_SIZE + MASK_BLK_SZ * n
        q = FPGA_DATA_SIZE + MASK_BLK_SZ * (n + 1) - 16
        print(m, n, fpga_data[p:p+16], fpga_data[q:q+16])
    time.sleep(0.1)

In [ ]:
mask_ptr = cast(c_void_p(fpga_data.ctypes.data + FPGA_DATA_SIZE), POINTER(c_char))

In [ ]:
for n in range(30):
    p = MASK_BLK_SZ * n
    print(np.frombuffer(mask_ptr[p:p+128], dtype='u1'))

In [ ]:
p = MASK_BLK_SZ * 4
vhex(np.frombuffer(mask_ptr[p:p+512], dtype='u8'))

In [ ]:
counts = bytes(bin(x).count("1") for x in range(256))

In [ ]:
counts[int.from_bytes(mask_ptr[0], 'little')]

In [ ]:
nz = 0

for p in range(MASK_BLK_SZ * 4, MASK_BLK_SZ * 5):
    nz += counts[int.from_bytes(mask_ptr[p], 'little')]

nz, MASK_BLK_SZ * 8

In [ ]:
blk_ptr[:16]

In [ ]:
hdr_id